# 이미지 분석 고도화 및 Use Case 확대 (11BT 공정 VGG16Net 카메라 적용, 엄선현)

In [ ]:
import PySpin
import cv2
import matplotlib.pyplot as plt
import datetime
import numpy as np
import time

In [3]:
from tensorflow.keras import applications, optimizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dropout, Flatten, Dense, Input
import tensorflow as tf

In [4]:
input_tensor = Input(shape=(224,224,3))
model = applications.VGG16(include_top=False,
                           input_tensor=input_tensor)
new_model = Sequential()
for l in model.layers:
    new_model.add(l)
#불러온 모델에 레이어를 붙일 수가 없어 새로운 모델을 선언하여 똑같은 vgg모델을 만드는 코드
new_model.add(Flatten()) # 1차원의 벡터로 펼쳐주는 역할
new_model.add(Dense(4096, activation='relu'))
new_model.add(Dense(4096, activation='relu'))
new_model.add(Dense(2, activation='softmax')) #출력값을 2개로 softmax함수로 활성화하는 코드

In [ ]:
new_model.load_weights(r"C:\um\new11bt/lbp_vgg16_test.h5") #가중치를 로드하는 코드

In [ ]:
cut_line = lambda x : x[450:550:,600:700]

In [2]:
system = PySpin.System.GetInstance()
cam_list = system.GetCameras()
#for i, cam in enumerate(cam_list):
#    print(i,cam)


In [ ]:
system = PySpin.System.GetInstance()

a_detected = []  #불량 데이터를 담을 공간
n_detected = []  #정상 데이터를 담을 공간

Width = 960
Height = 960


cam_list = system.GetCameras()
for i, cam in enumerate(cam_list):
    print(i,cam)
# 따라서 cam1에는 첫번째 카메라를 담음.
    cam1 = cam_list[0]

#첫번째 카메라
    cam1.Init()
    #cam1.AcquisitionMode.SetValue(PySpin.AcquisitionMode_Continuous) 카메라 세팅 설정하는 부분
    #cam1.PixelFormat.SetValue(PySpin.PixelFormat_BGR8) 카메라 세팅 설정하는 부분
    cam1.BeginAcquisition()


    while(True):
        
        # 1번 카메라
        
        image_result1 = cam1.GetNextImage()
        image_converted1 = image_result1.Convert(PySpin.PixelFormat_BGR8)
        #  Convert the Image object to RGB array
        width = image_result1.GetWidth()
        height = image_result1.GetHeight()
        rgb_array1 = image_converted1.GetData()
        rgb_array1 = rgb_array1.reshape(height, width, 3)
        image1 = cv2.resize(rgb_array1, (Width,Height))
        
        now_line = cut_line(frame)
        now_px = np.mean(now_line.flatten())
        
        if cnt>1:
    
            center = cut_line(frame)
            #now_px - pre_px > 10 회색 부분의 픽셀값의 평균과 이전 프레임의 회색 부분의 픽셀 값의 차이가 10보다 크고,
            #가온데 부분(이미지의 [450:550:,600:700]부분)의 픽셀 값이 100보다 크고,
            #이전의 캡쳐 시점에서 0.2초가 지났을 때 캡쳐 진행
            if ((now_px - pre_px) > 10) & (np.mean(center)>100)&(time.time()-t > 0.2) :
                f_cnt=0
                print("detected")
                print(f_cnt)

                t = time.time()
            if f_cnt == 8:
                #detected.append(frame2)
                im = image1[20:620,370:950]  #불량검출할 제품영역 지정
                im = cv2.resize(im,(224,224))
                im = im.reshape(1,224,224,3)
                im = im / 255 #정규화
                if np.argmax(new_model.predict(im)) == 1:
                    print("불량")
                    a_detected.append(image1[20:620,370:950])
                else:
                    print("정상")
                    n_detected.append(image1[20:620,370:950])
            
        f_cnt+=1
        
        pre_px = now_px
        cnt+=1

        
        cv2.imshow("Cam1",image1)

        
        if cv2.waitKey(1) & 0xFF==ord("q"):
            video1.release()
            video2.release()
            break
    
    cam1.EndAcquisition()
    
    cv2.destroyAllWindows()
    